# Image preprocessing

>>> Dataset TFRECORDS 256x256 with DULLRAZOR available here: [dataset](https://www.kaggle.com/antocad/melanoma-256x-dullrazor)

* 1st step: remove hairs with Dull Razor filter (it works but need an improvement) ([ref](https://www.sciencedirect.com/science/article/abs/pii/S0010482597000206?via%3Dihub))
* 2nd step: remove the noise with Median Filtering
* 3rd step: Discovering filters, Applying Neuron Engineer's method    ([Notebook](https://www.kaggle.com/nxrprime/siim-d3-eda-augmentations-and-resnext))

* Next step: Lesion segmentation

In [ ]:
import os,cv2,re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf

In [ ]:
TFREC = '../input/melanoma-256x256'

files_train = np.sort(np.array(tf.io.gfile.glob(TFREC + '/train*.tfrec')))
files_test  = np.sort(np.array(tf.io.gfile.glob(TFREC + '/test*.tfrec')))

In [ ]:
ds = tf.data.TFRecordDataset(files_train).shuffle(42)

In [ ]:
def read_tfrecord(example):
    tfrec_format = {
        'image'                        : tf.io.FixedLenFeature([], tf.string),
        'image_name'                   : tf.io.FixedLenFeature([], tf.string),
        'patient_id'                   : tf.io.FixedLenFeature([], tf.int64),
        'sex'                          : tf.io.FixedLenFeature([], tf.int64),
        'age_approx'                   : tf.io.FixedLenFeature([], tf.int64),
        'anatom_site_general_challenge': tf.io.FixedLenFeature([], tf.int64),
        'diagnosis'                    : tf.io.FixedLenFeature([], tf.int64),
        'target'                       : tf.io.FixedLenFeature([], tf.int64)
    }           
    example = tf.io.parse_single_example(example, tfrec_format)
    return example['image'], example['target']

In [ ]:
def image_decode(img):
    img,label = read_tfrecord(img)
    img = tf.image.decode_jpeg(img, channels=3)
    return img,label

In [ ]:
def image_preprocessing(img):
    
    plt.imshow(img)
    plt.show()
    
    #removing hairs
    img = dullrazor(img)
    #denoising
    img = cv2.medianBlur(img, 3)
    #filters
    #CALL A FILTER METHOD HERE: BENGRAHAM for example
 
    return img

In [ ]:
def dullrazor(img, lowbound=15, showimgs=True, filterstruc=3, inpaintmat=3):
    #grayscale
    imgtmp1 = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    #applying a blackhat
    filterSize =(filterstruc, filterstruc)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, filterSize) 
    imgtmp2 = cv2.morphologyEx(imgtmp1, cv2.MORPH_BLACKHAT, kernel)

    #0=skin and 255=hair
    ret, mask = cv2.threshold(imgtmp2, lowbound, 255, cv2.THRESH_BINARY)
    
    #inpainting
    img_final = cv2.inpaint(img, mask, inpaintmat ,cv2.INPAINT_TELEA)
    
    if showimgs:
        print("_____DULLRAZOR_____")
        plt.imshow(imgtmp1, cmap="gray")
        plt.show()
        plt.imshow(imgtmp2, cmap='gray')
        plt.show()
        plt.imshow(mask, cmap='gray')
        plt.show()
        plt.imshow(img_final)
        plt.show()
        print("___________________")

    return img_final

In [ ]:
def view_images_bengraham(image):
        image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
        image = cv2.resize(image, (256, 256))
        image = cv2.addWeighted ( image,4, cv2.GaussianBlur( image , (0,0) , 256/10) ,-4 ,128)
        plt.imshow(image, cmap=plt.cm.bone)
        plt.show()

In [ ]:
def view_images_neuronengineer(image):
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (256, 256))
        image = cv2.addWeighted ( image,4, cv2.GaussianBlur( image , (0,0) , 10) ,-4 ,128)
        plt.imshow(image, cmap=plt.cm.bone)
        plt.show()

In [ ]:
def crop_image_from_gray(img,tol=7):
    if img.ndim ==2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img>tol
        
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0): # image is too dark so that we crop out everything,
            return img # return original image
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
            img = np.stack([img1,img2,img3],axis=-1)
        return img
    
def circle_crop(img, sigmaX=10):   
    """
    Create circular crop around image centre    
    """    
    
    img = crop_image_from_gray(img)    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    height, width, depth = img.shape    
    
    x = int(width/2)
    y = int(height/2)
    r = np.amin((x,y))
    
    circle_img = np.zeros((height, width), np.uint8)
    cv2.circle(circle_img, (x,y), int(r), 1, thickness=-1)
    img = cv2.bitwise_and(img, img, mask=circle_img)
    img = crop_image_from_gray(img)
    img=cv2.addWeighted ( img,4, cv2.GaussianBlur( img , (0,0) , sigmaX) ,-4 ,128)
    return img 

def view_images_crop(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (256, 256))
    image= circle_crop(image)
    plt.imshow(image, cmap=plt.cm.bone)
    plt.show()

In [ ]:
tmp = ds.take(1)
tmp = tmp.map(lambda img: image_decode(img))

for img,label in tmp.as_numpy_iterator():
    img = image_preprocessing(img)
    view_images_bengraham(img)
    view_images_neuronengineer(img)
    view_images_crop(img)

### (useless part)

def my_test(img):
    def test_fn(img):
        return image_preprocessing(img.numpy())
    def call_test(img):
        return tf.py_function(test_fn, [img], tf.uint8)
    return call_test(img)

ds = ds.map(lambda img: image_decode(img))
ds = ds.map(lambda img, label: (tf.py_function(my_test, [img], tf.uint8),label))